## Hyperparameter Tuning with scikit-learn

### Imports

In [6]:
from sklearn.experimental import enable_halving_search_cv

from sklearn.model_selection import (train_test_split,
                                     KFold,
                                     cross_val_score,
                                     GridSearchCV,
                                     RandomizedSearchCV,
                                     HalvingGridSearchCV,
                                     HalvingRandomSearchCV)
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import seaborn as sns
import pandas as pd
import numpy as np

## Data Loading and Preprocessing

In [7]:
healthexp = sns.load_dataset('healthexp')
healthexp = pd.get_dummies(healthexp)
X = healthexp.drop('Life_Expectancy', axis=1)
y = healthexp['Life_Expectancy']
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [8]:
estimator = RandomForestRegressor(random_state=42)
cv = KFold(n_splits=3, shuffle=True, random_state=42)

In [9]:
def get_score(model, X_test, y_test):
    y_pred = model.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)

    print('Mean Absolute Error:', mae)
    print('Mean Squared Error:', mse)
    print('Root Mean Squared Error:', rmse)
    print('R^2 Score:', r2)
    return mae, mse, rmse, r2

### Let's check raw estimator performance

In [10]:
estimator.fit(X_train, y_train)

get_score(estimator, X_test, y_test)

Mean Absolute Error: 0.274527272727264
Mean Squared Error: 0.12436518181817355
Root Mean Squared Error: 0.35265447936779926
R^2 Score: 0.9898132982462418


(0.274527272727264,
 0.12436518181817355,
 np.float64(0.35265447936779926),
 0.9898132982462418)

### Let's use RandomizedSearchCV

In [11]:
search = RandomizedSearchCV(
    estimator=estimator,
    param_distributions={
        'n_estimators': np.arange(100, 1001),
        'max_depth': np.arange(5, 51),
        'min_samples_split': np.arange(2, 33),
        'min_samples_leaf': np.arange(1, 33),
    },
    n_iter=50,
    scoring='neg_mean_squared_error',
    cv=cv,
    n_jobs=-1,
    random_state=42,
    verbose=1
)

search.fit(X_train, y_train)


get_score(search.best_estimator_, X_test, y_test)

Fitting 3 folds for each of 50 candidates, totalling 150 fits
Mean Absolute Error: 0.48860659652982497
Mean Squared Error: 0.3224805251928763
Root Mean Squared Error: 0.5678736877095788
R^2 Score: 0.973585750581397


(0.48860659652982497,
 0.3224805251928763,
 np.float64(0.5678736877095788),
 0.973585750581397)

### Now, GridSearchCV

In [12]:
search = GridSearchCV(
    estimator=estimator,
    param_grid={
        'n_estimators': [100, 300, 500, 700, 1000],
        'max_depth': [5, 15, 25, 35, 50],
        'min_samples_split': [2, 8, 16, 24, 32],
        'min_samples_leaf': [1, 8, 16, 24, 32],
    },
    scoring='neg_mean_squared_error',
    cv=cv,
    n_jobs=-1,
    verbose=1
)

search.fit(X_train, y_train)

get_score(search.best_estimator_, X_test, y_test)

Fitting 3 folds for each of 625 candidates, totalling 1875 fits
Mean Absolute Error: 0.27023272727271863
Mean Squared Error: 0.11658027490910514
Root Mean Squared Error: 0.341438537527774
R^2 Score: 0.990450956823217


(0.27023272727271863,
 0.11658027490910514,
 np.float64(0.341438537527774),
 0.990450956823217)

In [ ]:
## Now, HalvingRandomSearchCV

In [13]:
search = HalvingRandomSearchCV(
    estimator=estimator,
    param_distributions={
        'n_estimators': np.arange(100, 1001),
        'max_depth': np.arange(5, 51),
        'min_samples_split': np.arange(2, 33),
        'min_samples_leaf': np.arange(1, 33),
    },
    n_candidates='exhaust',
    factor=3,
    scoring='neg_mean_squared_error',
    cv=cv,
    n_jobs=-1,
    random_state=42,
    verbose=1
)

search.fit(X_train, y_train)

get_score(search.best_estimator_, X_test, y_test)

n_iterations: 4
n_required_iterations: 4
n_possible_iterations: 4
min_resources_: 6
max_resources_: 219
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 36
n_resources: 6
Fitting 3 folds for each of 36 candidates, totalling 108 fits
----------
iter: 1
n_candidates: 12
n_resources: 18
Fitting 3 folds for each of 12 candidates, totalling 36 fits
----------
iter: 2
n_candidates: 4
n_resources: 54
Fitting 3 folds for each of 4 candidates, totalling 12 fits
----------
iter: 3
n_candidates: 2
n_resources: 162
Fitting 3 folds for each of 2 candidates, totalling 6 fits
Mean Absolute Error: 0.7337170837913145
Mean Squared Error: 0.7773680901548147
Root Mean Squared Error: 0.8816848020436865
R^2 Score: 0.936326094076127


(0.7337170837913145,
 0.7773680901548147,
 np.float64(0.8816848020436865),
 0.936326094076127)

## Finally, HalvingGridSearchCV

In [14]:
search = HalvingGridSearchCV(
    estimator=estimator,
    param_grid={
        'n_estimators': [100, 300, 500, 700, 1000],
        'max_depth': [5, 15, 25, 35, 50],
        'min_samples_split': [2, 8, 16, 24, 32],
        'min_samples_leaf': [1, 8, 16, 24, 32],
    },
    factor=3,
    scoring='neg_mean_squared_error',
    cv=cv,
    n_jobs=-1,
    verbose=1
)

search.fit(X_train, y_train)

get_score(search.best_estimator_, X_test, y_test)

n_iterations: 4
n_required_iterations: 6
n_possible_iterations: 4
min_resources_: 6
max_resources_: 219
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 625
n_resources: 6
Fitting 3 folds for each of 625 candidates, totalling 1875 fits
----------
iter: 1
n_candidates: 209
n_resources: 18
Fitting 3 folds for each of 209 candidates, totalling 627 fits
----------
iter: 2
n_candidates: 70
n_resources: 54
Fitting 3 folds for each of 70 candidates, totalling 210 fits
----------
iter: 3
n_candidates: 24
n_resources: 162
Fitting 3 folds for each of 24 candidates, totalling 72 fits
Mean Absolute Error: 0.2687927272726473
Mean Squared Error: 0.11476457272721334
Root Mean Squared Error: 0.3387692027431262
R^2 Score: 0.9905996802547288


(0.2687927272726473,
 0.11476457272721334,
 np.float64(0.3387692027431262),
 0.9905996802547288)